In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../scripts")
import util
import camera

# from sklearn.cluster import DBSCAN

np.set_printoptions(precision=4)
!pwd

/home/zpyang/git/visnet/src/visnet/notebooks


In [2]:
def msmt_association(measurements, map_hypothesis):
    """
    Parameters:
    -----------
    measurements: shape: (n_measurements, 2+1)
        From the m-th camera, 
    map_hypothesis: shape: (n_targets, 2+1)
        State hypothesis on cameras at t-1 of all tracked objects, 

    RETURN:
    msmt_m: 
        Associated measurements of m-th camera, order in the same way as the targets. |msmt_m[i] <-> tracks[i]|
    """

    msmt_m = []
    for hi in map_hypothesis:
        w = util.squared_error(measurements, hi,sigma=200)
        argmax_w = np.argmax(w)
        msmt_m.append(measurements[argmax_w])
        # measurements = np.delete(measurements, argmax_w, axis=0)
    
    return np.array(msmt_m)

def observe_fn(cam_group, particles):
        """
        Parameters:
            @ particles : all particles of a track
        Returns:
            @ hypothesis from x using the camera projection matrix
            This returns list of arrays which corresponds to hypothesis of particles on each camera
            |hypo[i] <-> camera[i]|
        """

        pos = particles[:, 0:3]
        labels = particles[:, -1]
        hypothesis = cam_group.get_group_measurement(pos, labels)

        return hypothesis

def prior_fn(n_particles, spawn_regions=None):
        if spawn_regions is not None:
            n_spawn = spawn_regions.shape[0]
            for region in spawn_regions:
                pos = np.random.normal(region, 2*np.ones(3), (n_particles, 3))
                vel = np.random.normal(np.zeros(3), np.ones(1), (n_particles, 3))
                # label = np.random.randint(0, 2, (n,1))
                particles = np.hstack([pos, vel])
            return particles
        else:
            pos = np.random.uniform(low=[-20, -20, 5], high=[20, 20, 25], size=(n_particles,3))
            vel = np.random.uniform(low=np.zeros(3), high=np.ones(3), size=(n_particles,3))

            # label = np.random.randint(0, 2, (n,1))
            particles = np.hstack([pos, vel])
            return particles

def weight_fn(msmt, hypo, sigma, verbose=False):
        """
        Iterate through each camera, update weights
        @ msmt: [{yj}_0, {yj}_1, ...{yj}_n]    n cameras
        @ hypo: [bearing_0, bearing_1, ....bearing_n]   n_cameras
        """

        n_particles = hypo[0].shape[0]
        weights = np.ones(n_particles)

        for zi, hi in zip(msmt, hypo):
            if zi[0] < 0: # got no measurements so weights are unchanged
                wi = np.ones(n_particles)
            wi = util.squared_error(hi, zi, sigma=sigma)
            weights *= wi

        return weights / np.sum(weights)

def resample(n_eff, particles, weights, resample_fn):
    if n_eff < n_eff_thresh:
        indices = resample_fn(weights)
        particles = particles[indices, :]
        weights = np.ones(n_particles) / n_particles
    
    return particles, weights
        
dt = 1/10
A = np.block([
    [np.eye(3), dt*np.eye(3)],
    [np.zeros((3,3)), np.eye(3)]
])
A = np.block([
    [A, np.zeros((6,1))],
    [np.zeros((1,6)), 1]
])

def dynamics_d(x):
    """
    Discrete dynamics, last state is the target label
    """
    n, d = x.shape
    w = np.zeros((d, n))
    w[3:6] = np.random.normal(0, 0.5, (3,n))
    x_1 = A @ x.T + w

    return x_1.T


In [3]:
def target_traj_circle(t):
    x = 15*np.sin(2 * np.pi * 0.01 * t / 3)
    y = 10*np.cos(2 * np.pi * 0.01 * t / 3)
    z = 20
    return np.array([x,y,z])

def target_traj_straight(t, start, finish, tf):
    max_dist = np.linalg.norm(start-finish)
    v_max = (finish-start) / tf
    # x = -15 + t * 0.1
    # y = 0
    # z = 20
    return start + v_max * t

def target_stationary(start):
    return start

In [4]:
class Track:
    """
    This is a data structure for representing tracked targets
    """
    def __init__(self, n_particles: int, x0=None, label=None, n_eff_thresh=1):
        self.n_particles = n_particles
        self.map_state = None # MAP estimate, particle with the max weight
        self.mean = None
        self.cov = None
        self.weights = np.ones(n_particles)
        self.particles = None
        self.label = label
        self.trajectory = []
        self.n_eff_thresh = n_eff_thresh
        
        self.cam_group = None
        self.hypothesis = None
        self.map_hypo = None # MAP estimate of pixel locations on different cameras

        if x0 is not None:
            x0 = x0 + np.random.normal(np.zeros(3), np.ones(3))
            pos = np.random.normal(x0, [1,1,1], size=(n_particles, 3))
            vel = np.random.normal([0,0,0], [1,1,1], size=(n_particles, 3))
            labels = np.ones((n_particles,1)) * label
            self.particles = np.hstack([pos, vel, labels])
            self.map_state = x0
        else:
            pos = np.random.uniform([-20,-20,0], [20,20,25], size=(n_particles, 3))
            vel = np.random.normal([0,0,0], [1,1,1], size=(n_particles, 3))
            self.particles = np.hstack([pos, vel])
    
    def set_map_hypo(self, map_hypo):
        self.map_hypo = map_hypo

    def set_map_state(self, map_state):
        self.map_state = map_state
    
    def update(self, weights, particles, resample_fn):
        """
        Update track after particle filter update
        """
        self.particles = particles
        self.weight_sum = np.sum(weights)
        self.weights = weights / self.weight_sum
        # mean and cov
        self.mean_state = np.sum(self.particles.T * self.weights, axis=-1).T
        self.cov_state = np.cov(self.particles, rowvar=False, aweights=self.weights)
        
        # Maximum-a-posterior MAP
        argmax_weight = np.argmax(self.weights)
        self.map_state = self.particles[argmax_weight]
        self.trajectory.append(self.map_state[0:3])
        
        self.n_eff = (1.0 / np.sum(self.weights ** 2)) / self.n_particles
        
        if self.n_eff < self.n_eff_thresh:
            indices = resample_fn(weights)
            self.particles = self.particles[indices, :]
            self.weights = np.ones(self.n_particles) / self.n_particles
        
        return particles, weights
    
    def set_cam_group(self, cam_group: camera.CamGroup):
        self.cam_group = cam_group

In [5]:
class ParticleFilter():
    """
    unknown number of targets
    """
    def __init__(self,
                 n_eff_thresh=1,
                 sigma=150,
                 resample_proportion=None,
                 verbose=False,
                ):
        self.n_eff_thresh = n_eff_thresh
        self.resample_proportion = resample_proportion or 0.0
        self.verbose = verbose
        self.tracks = []

    def update(self, track: Track, observation=None):
        particles = self.dynamics_d(track.particles)
        
        hypothesis = observe_fn(track.cam_group, particles)
        hypothesis = hypothesis.reshape(track.n_particles, track.cam_group.n_cams * 2)

        weights = weight_fn(observation, hypothesis)

        return weights

        # n_eff = (1.0 / np.sum(weights ** 2)) / track.n_particles
        # self.weight_entropy = np.sum(self.weights * np.log(self.weights, where=self.weights!=0))|

In [18]:
cam_param = [642.0926, 642.0926, 1000.5, 1000.5,0]
# [x y z roll pitch yaw]
cam_poses = np.array([
    [20, 20, 12, 0, 0, -2.2],
    [20, -20, 12, 0, 0, 2.2],
    [-20, -20, 12, 0, 0, 0.7],
    [-20, 20, 12, 0, 0, -0.7],
])

cam_group = camera.CamGroup(cam_param, cam_poses[0:4,:])

start = np.array([-18, 5, 15])
end = np.array([20, 5, 15])

n_targets = 2
tracks = []

t_final = 300

verbose = False
spawn_region = start
n_particles = 500

# pf = ParticleFilter(
#     n_eff_thresh=1,
#     sigma=150,
#     resample_proportion=0
# )

x0_1 = np.array([-20, 4, 20])
x0_2 = np.array([20, -4, 20])
x0_3 = np.array([5, 20, 15])

n_eff_thresh = 0.8
track1 = Track(n_particles, x0=x0_1, label=1, n_eff_thresh=n_eff_thresh)
track2 = Track(n_particles, x0=x0_2, label=1, n_eff_thresh=n_eff_thresh)

tracks = [track1, track2]

state_est1 = []
state_est2 = []

for t in range(t_final):
    target_1 = target_traj_straight(t, x0_1, x0_1+[30,0,0], t_final)
    target_2 = target_traj_straight(t, x0_2, x0_2+[-30,0,0], t_final)

    # target_2 = target_traj_circle(t)
    # target_1 = target_stationary(x0_1)
    # target_2 = target_stationary(x0_2)
    targets = np.vstack([target_1, target_2])
    
    if t > 100:
        target_3 = target_traj_straight(t-100, x0_3, x0_3+[0,30,0], t_final-50)
        targets = np.vstack([targets, target_3])

    #each camera can give multiple measurements
    z = cam_group.get_group_measurement(targets, labels=[1,1, 2])

    map_states = [track.map_state[0:3] for track in tracks]
    map_hypo = cam_group.get_group_measurement(map_states, labels=[1,2])

    z_a = []
    for z_m, map_hypo_m in zip(z, map_hypo):
        z_a_m = msmt_association(z_m, map_hypo_m)
        z_a.append(z_a_m)

    for i, track in enumerate(tracks):
        msmt = [z_a_m[i] for z_a_m in z_a]
        particles = dynamics_d(track.particles)
        hypo = observe_fn(cam_group, track.particles)
        weights = weight_fn(msmt=msmt, hypo=hypo, sigma=150)
        track.update(weights, particles, util.systematic_resample)


In [19]:
fig = plt.figure("trajectory", figsize=(10,10))

ax = plt.axes(projection="3d")
traj_1 = np.array(track1.trajectory)
ax.plot3D(traj_1[:,0], traj_1[:,1], traj_1[:,2], marker='.', color='b')
traj_2 = np.array(track2.trajectory)
ax.plot3D(traj_2[:,0], traj_2[:,1], traj_2[:,2], marker='.', color='r')
ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.set_xlabel('x, m')
ax.set_ylabel('y, m')
ax.set_zlabel('z, m')

Text(0.5, 0, 'z, m')

In [10]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import cm
# Attaching 3D axis to the figure
fig = plt.figure("Weight visualization and Clustering", figsize=(20, 10))

ax1 = fig.add_subplot(1, 2, 1, projection="3d")
ax2 = fig.add_subplot(1, 2, 2, projection="3d")
ax1.view_init(30,0)

def update_frame(i):
    # NOTE: there is no .set_data() for 3 dim data...
    x = particles[i, :, 0]
    y = particles[i, :, 1]
    z = particles[i, :, 2]
    w = weights[i, :]
    
    ax1.cla()
    ax1.set_xlim3d(-25,25)
    ax1.set_ylim3d(-25,25)
    ax1.set_zlim3d(0,25)
    ax1.scatter(x, y, z, cmap=cm.turbo, c=w, s=2)
    ax1.set_xlabel('x, m')
    ax1.set_ylabel('y, m')
    ax1.set_zlabel('z, m')
    
    ax2.cla()
    ax2.set_xlim3d(-25,25)
    ax2.set_ylim3d(-25,25)
    ax2.set_zlim3d(0,25)
    ax2.set_xlabel('x, m')
    ax2.set_ylabel('y, m')
    ax2.set_zlabel('z, m')

    colors = [plt.cm.tab10(each) for each in np.linspace(0, 1, len(clusters_i))]
    for cluster, col in zip(clusters_i, colors):
        ax2.scatter(
            cluster[:, 0],
            cluster[:, 1],
            cluster[:, 2],
            "o",
            color=col,
        )
        
    return None


ani = FuncAnimation(
    fig, update_frame, t_final, fargs=[],
    interval=int(1000 / 10), blit=False)
# ani.save("../videos/particles.mp4")
# HTML(ani.to_html5_video())
# plt.close()

Traceback (most recent call last):
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 270, in process
    func(*args, **kwargs)
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/animation.py", line 991, in _start
    self._init_draw()
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/animation.py", line 1753, in _init_draw
    self._draw_frame(next(self.new_frame_seq()))
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/animation.py", line 1776, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/tmp/ipykernel_6319/1484573118.py", line 14, in update_frame
    x = particles[i, :, 0]
NameError: name 'particles' is not defined


In [15]:
print(state.shape)
plt.figure(figsize=(10,10))
ax = plt.axes(projection="3d")
ax.plot3D(state[:,0], state[:,1], state[:,2], marker='+', label='truth1', markersize=10)
# ax.plot3D(state[:,3], state[:,4], state[:,5], '+-', label='truth2')
ax.scatter(state_est[:,0], state_est[:,1], state_est[:,2], marker='x', label='est')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.view_init(60,-90)

plt.legend()

plt.figure(figsize=(10,8))
plt.subplot(3,1,1)
plt.plot(err1)
plt.title('error plot')
plt.xlabel('# of frames')
plt.ylabel('error[m]')

plt.subplot(3,1,2)
plt.plot(n_eff)
plt.title('n_eff')
plt.xlabel('# of frames')
plt.ylabel('n_eff')

plt.subplot(3,1,3)
plt.plot(state_est[:,-1])
plt.title('label')
plt.xlabel('# of frames')
plt.ylabel('label')
plt.show()

Traceback (most recent call last):
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 270, in process
    func(*args, **kwargs)
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/animation.py", line 991, in _start
    self._init_draw()
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/animation.py", line 1753, in _init_draw
    self._draw_frame(next(self.new_frame_seq()))
  File "/home/zpyang/miniconda3/envs/visnet/lib/python3.9/site-packages/matplotlib/animation.py", line 1776, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/tmp/ipykernel_6006/1484573118.py", line 14, in update_frame
    x = particles[i, :, 0]
NameError: name 'particles' is not defined


NameError: name 'state' is not defined